In [156]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder

In [157]:
cleaned_file = 'Otodom_3p_cleaned.csv'
df = pd.read_csv(cleaned_file)
print(df.shape)
df.head(6)

(117, 12)


,price,latitude,longitude,surface_area,num_of_room,floor,num_floors_in_building,finish_condition,form_of_property,balcony_garden_terrace,parking_space,heating
0,NaN,52.172792,20.994102,35.41,5,3.0,8.0,for finishing,full ownership,"Elevator, Terrace",garage/parking space,NaN
1,970000.0,52.182325,21.025885,48.55,3,7.0,15.0,for living,full ownership,Laundry,garage/parking space,municipal
2,629000.0,52.286913,20.940933,36.50,1,0.0,10.0,for living,full ownership,Garage,NaN,municipal
3,2480000.0,52.238050,21.029350,99.69,1,3.0,4.0,for living,cooperative ownership right to the premises,Garage,NaN,municipal
4,560000.0,52.446508,20.692522,50.00,4,1.0,5.0,NaN,NaN,Garage,garage/parking space,NaN
5,819000.0,52.201297,20.889940,60.00,2,1.0,7.0,NaN,full ownership,"Laundry, Balcony, Elevator",garage/parking space,municipal


In [158]:
from sklearn.preprocessing import MultiLabelBinarizer

# Step 1: Split the strings into lists
df['balcony_garden_terrace'] = df['balcony_garden_terrace'].apply(lambda x: x.split(', '))

# Step 2: Apply MultiLabelBinarizer
mlb = MultiLabelBinarizer()
one_hot_encoded = mlb.fit_transform(df['balcony_garden_terrace'])

# Create a DataFrame with the one-hot encoded columns
one_hot_encoded_df = pd.DataFrame(one_hot_encoded, columns=mlb.classes_, index=df.index)

# Step 3: Combine the one-hot encoded columns with the original DataFrame
df = pd.concat([df, one_hot_encoded_df], axis=1)

# Drop the original 'balcony_garden_terrace' column
df = df.drop(["balcony_garden_terrace"], axis=1)

# Display the resulting DataFrame
df.head(6)

,price,latitude,longitude,surface_area,num_of_room,floor,num_floors_in_building,finish_condition,form_of_property,parking_space,heating,Balcony,Elevator,Garage,Laundry,Terrace
0,NaN,52.172792,20.994102,35.41,5,3.0,8.0,for finishing,full ownership,garage/parking space,NaN,0,1,0,0,1
1,970000.0,52.182325,21.025885,48.55,3,7.0,15.0,for living,full ownership,garage/parking space,municipal,0,0,0,1,0
2,629000.0,52.286913,20.940933,36.50,1,0.0,10.0,for living,full ownership,NaN,municipal,0,0,1,0,0
3,2480000.0,52.238050,21.029350,99.69,1,3.0,4.0,for living,cooperative ownership right to the premises,NaN,municipal,0,0,1,0,0
4,560000.0,52.446508,20.692522,50.00,4,1.0,5.0,NaN,NaN,garage/parking space,NaN,0,0,1,0,0
5,819000.0,52.201297,20.889940,60.00,2,1.0,7.0,NaN,full ownership,garage/parking space,municipal,1,1,0,1,0


In [159]:
print(df['Balcony'].dtype)


int32


In [160]:
df['parking_space'] = df['parking_space'].apply(lambda x: 1 if x == 'garage/parking space' else 0).astype('int8')

In [161]:
df = pd.concat([df, pd.get_dummies(df['form_of_property'])], axis=1)
df = df.drop(['form_of_property'], axis=1)

In [162]:
df = pd.concat([df, pd.get_dummies(df['heating'])], axis = 1)
df = df.drop(['heating'], axis=1)

In [163]:
df.head(6)

,price,latitude,longitude,surface_area,num_of_room,floor,num_floors_in_building,finish_condition,parking_space,Balcony,...,Garage,Laundry,Terrace,cooperative ownership right to the premises,full ownership,boiler room,electric,gas,municipal,other
0,NaN,52.172792,20.994102,35.41,5,3.0,8.0,for finishing,1,0,...,0,0,1,False,True,False,False,False,False,False
1,970000.0,52.182325,21.025885,48.55,3,7.0,15.0,for living,1,0,...,0,1,0,False,True,False,False,False,True,False
2,629000.0,52.286913,20.940933,36.50,1,0.0,10.0,for living,0,0,...,1,0,0,False,True,False,False,False,True,False
3,2480000.0,52.238050,21.029350,99.69,1,3.0,4.0,for living,0,0,...,1,0,0,True,False,False,False,False,True,False
4,560000.0,52.446508,20.692522,50.00,4,1.0,5.0,NaN,1,0,...,1,0,0,False,False,False,False,False,False,False
5,819000.0,52.201297,20.889940,60.00,2,1.0,7.0,NaN,1,1,...,0,1,0,False,True,False,False,False,True,False


In [164]:
# df['finish_condition'] = df['finish_condition'].apply(lambda x:
#                                                       1 if x == 'for living'
#                                                       else 0.8 if x == 'for renovation'
#                                                       else 0.5 if pd.isnull(x)
#                                                       else 0 if x == 'for finishing'
#                                                       else 0.5)
# 'for living'
# 'for finishing'
# 'for renovation'
#  NaN

In [165]:
# Split the DataFrame into two parts based on whether 'price' is missing
df_without_price = df[df['price'].isna()]
df = df[df['price'].notna()]

X = df.drop('price', axis=1)
y = df['price']

In [169]:
df.sample(5)

,price,latitude,longitude,surface_area,num_of_room,floor,num_floors_in_building,finish_condition,parking_space,Balcony,...,Garage,Laundry,Terrace,cooperative ownership right to the premises,full ownership,boiler room,electric,gas,municipal,other
87,548000.0,52.390571,20.923325,52.90,1,5.0,9.0,for living,0,1,...,0,0,0,False,True,False,False,False,True,False
22,749000.0,52.132022,21.231838,75.00,4,2.0,2.0,for living,1,1,...,0,0,1,False,True,False,False,False,True,False
28,940000.0,52.328725,21.171824,121.00,2,1.0,2.0,for finishing,1,1,...,1,0,0,False,True,False,False,True,False,False
53,799000.0,52.190911,20.906492,53.10,5,3.0,6.0,for living,1,0,...,0,0,1,False,True,False,False,False,True,False
19,1590000.0,52.226743,21.039280,63.12,1,3.0,4.0,for living,0,0,...,1,0,1,False,True,False,False,False,True,False


In [166]:
df.columns

Index(['price', 'latitude', 'longitude', 'surface_area', 'num_of_room',
       'floor', 'num_floors_in_building', 'finish_condition', 'parking_space',
       'Balcony', 'Elevator', 'Garage', 'Laundry', 'Terrace',
       'cooperative ownership right to the premises', 'full ownership',
       'boiler room', 'electric', 'gas', 'municipal', 'other'],
      dtype='object')

## Hypothesis Testing
### Null Hypothesis (H0): A <U>will-be-ready-to-live</U> house is cheaper than a <U>ready-to-live</U> house.
### Alternative Hypothesis (H1): A <U>will-be-ready-to-live</U> house is NOT cheaper so buying it will give no significant benefit for the buyer.

In [173]:
price_living = df[df['finish_condition'] == 'for living']['price']
price_finishing = df[df['finish_condition'] == 'for finishing']['price']
print(price_living.shape)
print(price_finishing.shape)

(82,)
(14,)


In [180]:
import scipy as sp
SIGNIFICANCE_LEVEL = 0.05

t_stat, p_value = sp.stats.ttest_ind(price_living, price_finishing)
print(t_stat, p_value)

rejecting = p_value < SIGNIFICANCE_LEVEL
print("NULL Hypth. H0: A will-be-ready-to-live house is cheaper than a ready-to-live house.")
print(("" if rejecting else "Not ") + "possible to reject the null hypothesis")

1.663090142066532 0.09962526245987119
NULL Hypth. H0: A will-be-ready-to-live house is cheaper than a ready-to-live house.
Not possible to reject the null hypothesis
